In [1]:
# !wget https://huggingface.co/datasets/mesolitica/malaysia-hansard-qa/resolve/main/hansard-qa.json

In [3]:
import malaya

model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/translation-t5-small-standard-bahasa-cased')

2023-07-25 12:53:44.014772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 12:53:44.201822: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 12:53:45.116100: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 12:53:45.116382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [10]:
_ = model.cuda()
_ = model.model.eval()

In [12]:
def generate_noisy(text):
    g = model.generate([text],to_lang = 'pasar ms',
              max_length = 512, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)
    result = []
    for g_ in g:
        splitted = g_.split()
        splitted = [s for s in splitted if s[0] !='@']
        s = ' '.join(splitted)
        if len(s):
            result.append(s)
    return result

generate_noisy("Apakah makna nama 'Guinness'?")

['Apakah makna nama Guinness',
 "Ckp 'Guinness' tu?",
 '"Guinness" ni apa maknanya',
 'makna tu nama guaness ka?',
 "Nama 'Guinness' tu apa maknanya"]

In [2]:
with open('hansard-qa.json') as fopen:
    hansard = json.load(fopen)

In [13]:
len(hansard)

42440

In [16]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [22]:
import json
import random
from tqdm import tqdm

In [28]:
data_parlimen = []
for d in tqdm(hansard):
    try:
        t = d[0]
        a = d[1]
        j_a = json.loads(d[1])
        if random.random() > 0.5:
            q = t['original']
        else:
            q = t['cleaned']
        
        
        for r in j_a['soalan']:
            randint = random.randint(0, 3)
            if random.random() > 0.5:
                g = generate_noisy(r['soalan'])
                soalan = random.choice(g)
            else:
                soalan = r['soalan']
            
            if random.random() > 0.5:
                soalan = soalan.lower()
            
            if random.random() > 0.7:
                q_ = f"paragraph `{q}`, berdasarkan paragraph, jawab soalan `{soalan}`"
            else:
                q_ = f"paragraph `{q}`, {soalan}"
            
            data = {
                'prompt_input': template_malaya["prompt_no_input"],
                'input': q_,
                'output': r['jawapan']
            }
            data_parlimen.append(data)
        
        k = '({"soalan": [{`soalan`, `jawapan`}]})'
        p = f"""
paragraph, `{t['cleaned']}`
hasilkan 3 soalan dan jawapan khusus berdasarkan perenggan itu, kembalikan sebagai struktur JSON {k}
        """
        data = {
            'prompt_input': template_malaya["prompt_no_input"],
            'input': p,
            'output': a
        }
        data_parlimen.append(data)

    except Exception as e:
        print(e)
        pass

  5%|████▎                                                                                        | 1943/42440 [19:43<4:03:55,  2.77it/s]

Unterminated string starting at: line 12 column 17 (char 764)


  6%|█████▊                                                                                       | 2678/42440 [27:14<3:26:53,  3.20it/s]

Expecting ',' delimiter: line 13 column 109 (char 1293)


  7%|██████▌                                                                                      | 3013/42440 [30:53<9:06:08,  1.20it/s]

...

 95%|███████████████████████████████████████████████████████████████████████████████████████▌    | 40390/42440 [8:10:12<27:18,  1.25it/s]

Expecting ',' delimiter: line 4 column 26 (char 47)


 95%|███████████████████████████████████████████████████████████████████████████████████████▊    | 40519/42440 [8:11:50<12:17,  2.61it/s]

Unterminated string starting at: line 13 column 18 (char 1779)


 97%|█████████████████████████████████████████████████████████████████████████████████████████▎  | 41228/42440 [8:21:05<25:41,  1.27s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
with open('prepared-parliament-qa.jsonl', 'w') as fopen:
    for l in data_parlimen:
        fopen.write(f'{json.dumps(l)}\n')

In [30]:
!head -n 3 prepared-parliament-qa.jsonl

{"prompt_input": "Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n", "input": "paragraph `\n22                                                            DR.30.10.2018                                                                                                                          \n\n \ndan sebagainya. Cukai-cukai yang didapati daripada minuman keras, alkohol dan sebagainya \nmasuk kepada kerajaan, kerajaan bayar gaji kita balik, itu pun haram pada kita.  [Disampuk]  \n\nTimbalan Yang di-Pertua [Dato\u2019 Mohd Rashid Hasnon]: Yang Berhormat Padang \nSerai, sila ajukan soalan. \n\nTuan Karupaiya Mutusami [Padang Serai]: Saya bertanya soalan pada pihak \nkementerian, bolehkah samsu dihapuskan terus, tidak mahu ada samsu dalam negara kita. \n[Dewan tepuk] Terima kasih. \n\nTuan Che Alias bin Hamid [Kemaman]: Sokong, sokong. \n\n\u25a01150 \n\nTuan Chong Chieng Jen: Itu satu isu atau s